In [1]:
import antigranular as ag
session = ag.login("yveemYmiUV93QmAJlwv3dQ9DvbISuTrJ", "CadbiOM3+dIU+7rfUIPCryYNtN92xLC0UovJA65/z394+l6dInUBrlueleJlqpEF", competition = "Sandbox for Harvard OpenDP Hackathon")

C:\Users\taori\Desktop\users\AntigranularClient\antigranular\client.py:143: UserWarning: Antigranular client version 0.2.14 not in supported clients list shared by the server, please update antigranular client to the latest version.
  warnings.warn(


Output: Dataset "Health Organisation Dataset" loaded to the kernel as health_organisation_dataset

Output: Dataset "Flight Company Dataset" loaded to the kernel as flight_company_dataset

Connected to Antigranular server session id: 3fc551fb-5ccb-4bd3-81c8-fa14059a794a, the session will time out if idle for 26 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [2]:
%%ag
import time

ImportError: Restricted import: time is not allowed


In [3]:
%%ag
health = health_organisation_dataset_for_sandbox
flight = flight_company_dataset_for_sandbox

In [4]:
%%ag
# Printing the column names using ag_print
ag_print(f"Flight Dataset \n {flight.columns} \n")
ag_print(f"Health Datset \n {health.columns}")

Flight Dataset 
 ['flight_number', 'flight_date', 'flight_from', 'flight_to', 'passenger_firstname', 'passenger_lastname', 'passenger_date_of_birth'] 

Health Datset 
 ['patient_firstname', 'patient_lastname', 'patient_date_of_birth', 'covidtest_date', 'covidtest_result', 'patient_address']



In [5]:
%%ag
# Getting the differentially private count
ag_print(f"Total health records : {health['patient_firstname'].count(eps=0.1)}")
ag_print(f"Total flight records :  {flight['passenger_firstname'].count(eps=0.1)}")

Total health records : 56861

Total flight records :  37208



In [6]:
%%ag
# Lets remove those passenger records who tested negative.
health['covidtest_result'] = health['covidtest_result'].where(health['covidtest_result'] == 'positive')
health = health.dropna()

In [7]:
%%ag
ag_print(f"Total covid positive health records : {health['patient_firstname'].count(eps=0.1)}")

Total covid positive health records : 1256



In [8]:
%%ag
import op_recordlinkage as rl
# A full indexing is a complete cartesian product
indexer  = rl.Index()
indexer.full()

Lets index both datasets based on the date of birth. However , you must take care of the format of the dates so that they both are in similar fashion before you index both datasets against them.

**click [here](https://recordlinkage.readthedocs.io/en/latest/ref-index.html) to learn about more indexing algorithms.**

We currently support:
- Full
- Block
- SortedNeighbor
- Random


In [9]:
%%ag
import op_recordlinkage as rl
indexer = rl.Index()
indexer.block('passenger_date_of_birth','patient_date_of_birth')
candidate_links = indexer.index(flight,health)

In [10]:
%%ag
# total number of links based on this indexing choice.
ag_print(candidate_links.count(eps=0.1))

5678



### Setting Comparison Rules

Once the candidate links are formed , we can set compare rules against them to refine our linking process. In these rules , we can set a weight for each compare rule that we define.
  - Lets fuzzy match the firstnames and lastnames(thresholded) of the passenger. ( using default value of either 1 or 0 )
  - Allow links on the positive covid result happening atleast 14 before flight departure. ( with weight = 2 )
 
**click [here](https://recordlinkage.readthedocs.io/en/latest/ref-compare.html#recordlinkage.Compare) to learn about more compare rules.**

We currently support:
- String
- Numeric
- Exact
- Geo
- Date
- Custom Compares
    

In [11]:
%%ag
compare = rl.Compare()

# Using inbuilt string linking via fuzzy match. ( keeping threshold for last_name to get stronger links )
compare.string("passenger_firstname" , "patient_firstname" ,method='jarowinkler', label="firstname")
compare.string("passenger_lastname","patient_lastname" ,method='jarowinkler', threshold=0.9, label="lastname")


# Using a custom compare rule.
from datetime import datetime
def cmp(date_str1 : str , date_str2: str) -> int:
    # Convert date strings to datetime objects
    date1 = datetime.datetime.strptime(date_str1, "%Y-%m-%d")
    date2 = datetime.datetime.strptime(date_str2, "%Y-%m-%d")
    
    # Calculate the absolute difference in days
    days_apart = (date2 - date1).days
    # Check if the dates are within two weeks (14 days) apart
    if days_apart <= 14:
        return 2
    else:
        return 0

compare.custom(cmp,"flight_date","covidtest_date",label="date_cmp")


In [12]:
%%ag
features = compare.compute(candidate_links,flight,health)

In [13]:
%%ag
# Lets find out the average matching weights obtained based on the compare rules we set.
ag_print(f"Average weight : {features.sum(axis=1).mean(eps=0.1)}")

Average weight : 1.166672154293226



### Linking the datasets

Choosing a value = 3 for linking based on the obtained average. Our purpose of giving the custom_compare
a weight=2 was to prioritize it over the matching of first_names/last_names. Moreover choosing a value=3 as threshold
for matching will give us strong links based on the compare rules we have set.

In [14]:
%%ag
linked_df = compare.get_match(3)

ValueError: compute(pairs,x,x_link) should be called before invoking get_match() method.


In [15]:
%%ag
# Submitting the column containing the filtered set of airlines we should report regarding a covid passenger.
res = linked_df[["l_flight_number"]]
x = submit_predictions(res)

NameError: name 'linked_df' is not defined


In [16]:
test

NameError: name 'test' is not defined

In [17]:
session.terminate_session()

{'status': 'ok'}